# Use Custom Template Notebook

In [1]:
from snowflake.snowpark import Session

# Data Science Libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# create_temp_table warning suppresion
import warnings; warnings.simplefilter('ignore')

#Modeling Libs
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier

from xgboost import XGBClassifier
#from catboost import CatBoostClassifier

from joblib import dump, load
import requests

# Initialize Snowflake Session using Default connection

In [2]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [3]:
table_name = 'AUTO_INSURANCE_CLAIMS_DATA'

In [4]:
sf_df = my_session.sql("select * from {}".format(table_name))

In [5]:
df = sf_df.to_pandas()

In [6]:
df.head()

,MONTHS_AS_CUSTOMER,CUSTOMER_AGE,POLICY_NUMBER,POLICY_BIND_DATE,POLICY_STATE,POLICY_CSL,POLICY_DEDUCTABLE,POLICY_ANNUAL_PREMIUM,UMBRELLA_LIMIT,INSURED_ZIP,...,POLICE_REPORT_AVAILABLE,TOTAL_CLAIM_AMOUNT_PAID,INJURY_CLAIM,PROPERTY_CLAIM,VEHICLE_CLAIM,AUTO_MAKE,AUTO_MODEL,AUTO_YEAR,FRAUD_REPORTED,POLICY_BIND_DATE_CUSTOM
0,139,37,457069,17-08-2011,CT,250/500,1351,1589,16139812,609322,...,No Police Report Available,48182,538,20189,27455,Dodge,Neon,2005,No Fraud Reported,2011-08-17
1,292,52,429086,27-09-2010,CT,100/300,2216,2669,15406514,438830,...,Police Report Available,64579,11170,5093,48316,Audi,A3,1999,No Fraud Reported,2010-09-27
2,45,20,457562,13-01-2002,CT,100/300,929,1290,5629268,616164,...,No Police Report Available,32420,538,845,31037,Mercedes,E400,2014,No Fraud Reported,2002-01-13
3,49,67,471692,07-02-2012,CT,500/1000,699,970,18501194,445120,...,No Police Report Available,53780,7280,898,45602,Suburu,Legacy,2005,No Fraud Reported,2012-02-07
4,128,45,463009,18-06-2008,CT,100/300,1122,1384,16890664,442936,...,Police Report Available,94511,412,14545,79554,Ford,Escape,2013,No Fraud Reported,2008-06-18


In [7]:
insurance_claim =  df.copy()

In [8]:
insurance_claim = insurance_claim[['MONTHS_AS_CUSTOMER', 'CUSTOMER_AGE', 'POLICY_NUMBER',
       'POLICY_BIND_DATE', 'POLICY_STATE', 'POLICY_CSL', 'POLICY_DEDUCTABLE',
       'POLICY_ANNUAL_PREMIUM', 'UMBRELLA_LIMIT', 'INSURED_ZIP', 'INSURED_SEX',
       'INSURED_EDUCATION_LEVEL', 'INSURED_OCCUPATION', 'INSURED_HOBBIES',
       'INSURED_RELATIONSHIP', 'CAPITAL_GAINS', 'CAPITAL_LOSS',
       'INCIDENT_DATE', 'INCIDENT_TYPE', 'COLLISION_TYPE', 'INCIDENT_SEVERITY',
       'AUTHORITIES_CONTACTED', 'INCIDENT_STATE', 'INCIDENT_CITY',
       'INCIDENT_LOCATION', 'INCIDENT_HOUR_OF_THE_DAY', 'INCIDENT_TIME_OF_DAY',
       'NUMBER_OF_VEHICLES_INVOLVED', 'PROPERTY_DAMAGE', 'BODILY_INJURIES',
       'WITNESSES', 'POLICE_REPORT_AVAILABLE', 'TOTAL_CLAIM_AMOUNT_PAID',
       'INJURY_CLAIM', 'PROPERTY_CLAIM', 'VEHICLE_CLAIM', 'AUTO_MAKE',
       'AUTO_MODEL', 'AUTO_YEAR', 'FRAUD_REPORTED']]

In [9]:
insurance_claim.columns = insurance_claim.columns.str.lower()

In [10]:
insurance_claim.columns

Index(['months_as_customer', 'customer_age', 'policy_number',
       'policy_bind_date', 'policy_state', 'policy_csl', 'policy_deductable',
       'policy_annual_premium', 'umbrella_limit', 'insured_zip', 'insured_sex',
       'insured_education_level', 'insured_occupation', 'insured_hobbies',
       'insured_relationship', 'capital_gains', 'capital_loss',
       'incident_date', 'incident_type', 'collision_type', 'incident_severity',
       'authorities_contacted', 'incident_state', 'incident_city',
       'incident_location', 'incident_hour_of_the_day', 'incident_time_of_day',
       'number_of_vehicles_involved', 'property_damage', 'bodily_injuries',
       'witnesses', 'police_report_available', 'total_claim_amount_paid',
       'injury_claim', 'property_claim', 'vehicle_claim', 'auto_make',
       'auto_model', 'auto_year', 'fraud_reported'],
      dtype='object')

In [11]:
insurance_claim.rename(columns = {'total_claim_amount_paid': 'total_claim_amount'}, inplace=True)

In [12]:
insurance_claim=insurance_claim.replace("?",np.NaN)

In [13]:
df = insurance_claim.copy()

In [14]:
actual_inference = df.copy()
actual_inference.drop('fraud_reported', axis = 1, inplace=True)

In [15]:
actual_inference.head()

,months_as_customer,customer_age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,...,bodily_injuries,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year
0,139,37,457069,17-08-2011,CT,250/500,1351,1589,16139812,609322,...,0,3,No Police Report Available,48182,538,20189,27455,Dodge,Neon,2005
1,292,52,429086,27-09-2010,CT,100/300,2216,2669,15406514,438830,...,1,2,Police Report Available,64579,11170,5093,48316,Audi,A3,1999
2,45,20,457562,13-01-2002,CT,100/300,929,1290,5629268,616164,...,0,2,No Police Report Available,32420,538,845,31037,Mercedes,E400,2014
3,49,67,471692,07-02-2012,CT,500/1000,699,970,18501194,445120,...,2,1,No Police Report Available,53780,7280,898,45602,Suburu,Legacy,2005
4,128,45,463009,18-06-2008,CT,100/300,1122,1384,16890664,442936,...,0,1,Police Report Available,94511,412,14545,79554,Ford,Escape,2013


In [16]:
# dropping columns which are not necessary for prediction

to_drop = ['policy_number','policy_bind_date','policy_state','insured_zip','incident_location','incident_date',
           'incident_state','incident_city','insured_hobbies','auto_make','auto_model','auto_year']

df.drop(to_drop, inplace=True, axis=1)

In [17]:
df['incident_hour_of_the_day'] = df['incident_hour_of_the_day'].astype(str).astype(int)

In [18]:
df['incident_hour_of_the_day'].dtype

dtype('int64')

In [19]:
# separating the feature and target columns

X = df.drop('fraud_reported', axis = 1)
y = df['fraud_reported']

In [20]:
# extracting categorical columns
cat_df = X.select_dtypes(include = ['object'])

In [21]:
cat_df.columns

Index(['policy_csl', 'insured_sex', 'insured_education_level',
       'insured_occupation', 'insured_relationship', 'incident_type',
       'collision_type', 'incident_severity', 'authorities_contacted',
       'incident_time_of_day', 'property_damage', 'police_report_available'],
      dtype='object')

In [22]:
# printing unique values of each column
for col in cat_df.columns:
    print(f"{col}: \n{cat_df[col].unique()}\n")

policy_csl: 
['250/500' '100/300' '500/1000']

insured_sex: 
['MALE' 'FEMALE']

insured_education_level: 
['High School' 'MD' 'Masters' 'Associate' 'College' 'PhD' 'JD']

insured_occupation: 
['handlers-cleaners' 'armed-forces' 'prof-specialty' 'sales'
 'machine-op-inspct' 'priv-house-serv' 'adm-clerical' 'exec-managerial'
 'tech-support' 'farming-fishing' 'craft-repair' 'transport-moving'
 'protective-serv' 'other-service']

insured_relationship: 
['wife' 'unmarried' 'other-relative' 'husband' 'own-child' 'not-in-family']

incident_type: 
['Single Vehicle Collision' 'Parked Car' 'Multi-vehicle Collision'
 'Vehicle Theft']

collision_type: 
['Front Collision' 'Rear Collision' 'Side Collision'
 'Details not Available']

incident_severity: 
['Minor Damage' 'Major Damage' 'Total Loss']

authorities_contacted: 
['Ambulance' 'Other' 'None' 'Police' 'Fire']

incident_time_of_day: 
['Morning to Noon' 'Night Time' 'Afternoon Hours' 'Early Morning Hours']

property_damage: 
['Property Damage' '

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232025 entries, 0 to 232024
Data columns (total 28 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   months_as_customer           232025 non-null  int16 
 1   customer_age                 232025 non-null  int8  
 2   policy_csl                   232025 non-null  object
 3   policy_deductable            232025 non-null  int16 
 4   policy_annual_premium        232025 non-null  int16 
 5   umbrella_limit               232025 non-null  int32 
 6   insured_sex                  232025 non-null  object
 7   insured_education_level      232025 non-null  object
 8   insured_occupation           232025 non-null  object
 9   insured_relationship         232025 non-null  object
 10  capital_gains                232025 non-null  int32 
 11  capital_loss                 232025 non-null  int32 
 12  incident_type                232025 non-null  object
 13  collision_type

In [24]:
cat_df = pd.get_dummies(cat_df, drop_first = True)

In [25]:
num_df = df.select_dtypes(include = ['int16','int8','int32','float64','int64'])

In [26]:
num_df.columns

Index(['months_as_customer', 'customer_age', 'policy_deductable',
       'policy_annual_premium', 'umbrella_limit', 'capital_gains',
       'capital_loss', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'bodily_injuries', 'witnesses',
       'total_claim_amount', 'injury_claim', 'property_claim',
       'vehicle_claim'],
      dtype='object')

In [27]:
cat_df.columns

Index(['policy_csl_250/500', 'policy_csl_500/1000', 'insured_sex_MALE',
       'insured_education_level_College',
       'insured_education_level_High School', 'insured_education_level_JD',
       'insured_education_level_MD', 'insured_education_level_Masters',
       'insured_education_level_PhD', 'insured_occupation_armed-forces',
       'insured_occupation_craft-repair', 'insured_occupation_exec-managerial',
       'insured_occupation_farming-fishing',
       'insured_occupation_handlers-cleaners',
       'insured_occupation_machine-op-inspct',
       'insured_occupation_other-service',
       'insured_occupation_priv-house-serv',
       'insured_occupation_prof-specialty',
       'insured_occupation_protective-serv', 'insured_occupation_sales',
       'insured_occupation_tech-support',
       'insured_occupation_transport-moving',
       'insured_relationship_not-in-family',
       'insured_relationship_other-relative', 'insured_relationship_own-child',
       'insured_relationship

In [28]:
# combining the Numerical and Categorical dataframes to get the final dataset
X = pd.concat([num_df, cat_df], axis = 1)

In [29]:
X.columns = X.columns.str.replace('/', '_')
X.columns = X.columns.str.replace(' ', '_')
X.columns = X.columns.str.replace('-', '_')

In [30]:
# splitting data into training set and test set
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [31]:
num_df = X_train[['months_as_customer', 'policy_deductable', 'umbrella_limit',
       'capital_gains', 'capital_loss', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'bodily_injuries', 'witnesses', 'injury_claim', 'property_claim',
       'vehicle_claim']]

In [32]:
num_df_test = X_test[['months_as_customer', 'policy_deductable', 'umbrella_limit',
       'capital_gains', 'capital_loss', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'bodily_injuries', 'witnesses', 'injury_claim', 'property_claim',
       'vehicle_claim']]

In [33]:
# Scaling the numeric values in the dataset
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_data = scaler.fit_transform(num_df)

In [34]:
scaled_data_test = scaler.transform(num_df_test)

In [35]:
scaled_num_df = pd.DataFrame(data = scaled_data, columns = num_df.columns, index = X_train.index)
scaled_num_df.head()

,months_as_customer,policy_deductable,umbrella_limit,capital_gains,capital_loss,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,witnesses,injury_claim,property_claim,vehicle_claim
39504,0.944739,-0.449266,1.011348,1.416752,-1.469583,0.846248,0.748264,1.226661,-0.450491,0.887490,-0.961795,0.549074
58451,-0.536010,1.407415,-0.473157,1.257006,0.340418,-0.861121,-0.678195,0.002539,1.340891,0.543041,1.156190,-0.543485
92217,1.449291,0.450093,1.045881,1.121061,-0.634188,-0.348910,2.174724,-1.221582,-1.346182,-1.028705,2.156032,-0.388079
35705,0.209849,-1.569257,-0.086327,0.303659,-1.516100,-2.568489,-0.678195,0.002539,-1.346182,-0.367895,1.330972,-0.042845
231420,-0.568915,1.366280,0.452609,0.421654,-0.468113,1.358459,0.748264,0.002539,-1.346182,1.767265,0.070612,0.183953


In [36]:
scaled_num_df_test = pd.DataFrame(data = scaled_data_test, columns = num_df_test.columns, index = X_test.index)
scaled_num_df_test.head()

,months_as_customer,policy_deductable,umbrella_limit,capital_gains,capital_loss,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,witnesses,injury_claim,property_claim,vehicle_claim
12308,-1.161215,0.279944,0.171792,-0.090956,-1.537397,1.187722,0.748264,-1.221582,-0.450491,-1.030801,-0.466199,1.958182
2131,-1.237994,0.006958,1.224716,-1.334468,-1.010198,-1.202594,-0.678195,0.002539,-0.450491,1.172550,-0.948693,1.557237
13195,0.089195,1.358801,0.468307,0.202206,0.595308,1.358459,0.748264,1.226661,-1.346182,-0.064110,0.293350,-1.126926
86590,1.054424,-1.402848,-1.758848,0.993712,1.407002,1.358459,-0.678195,-1.221582,-0.450491,-1.002295,-0.961413,-1.135085
130719,-0.294702,1.310187,0.143353,0.185520,1.520488,0.675511,2.174724,-1.221582,-0.450491,-1.019343,0.754473,-0.076693


In [37]:
X_train.drop(columns = scaled_num_df.columns, inplace = True)

In [38]:
X_train = pd.concat([scaled_num_df, X_train], axis = 1)

In [39]:
X_test.drop(columns = scaled_num_df_test.columns, inplace = True)

In [40]:
X_test = pd.concat([scaled_num_df_test, X_test], axis = 1)

# Decision Tree

In [41]:
#from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)

DecisionTreeClassifier()

In [42]:
y_pred = dtc.predict(X_test)

In [43]:
dtc_train_acc = accuracy_score(y_train, dtc.predict(X_train))
dtc_test_acc = accuracy_score(y_test, y_pred)

print(f"Training accuracy of Decision Tree is : {dtc_train_acc}")
print(f"Test accuracy of Decision Tree is : {dtc_test_acc}")

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Training accuracy of Decision Tree is : 1.0
Test accuracy of Decision Tree is : 0.9940352026479563
[[ 8540   138]
 [  208 49121]]
                   precision    recall  f1-score   support

   Fraud Reported       0.98      0.98      0.98      8678
No Fraud Reported       1.00      1.00      1.00     49329

         accuracy                           0.99     58007
        macro avg       0.99      0.99      0.99     58007
     weighted avg       0.99      0.99      0.99     58007



In [44]:
from fosforml import register_model

In [45]:
X.columns

Index(['months_as_customer', 'customer_age', 'policy_deductable',
       'policy_annual_premium', 'umbrella_limit', 'capital_gains',
       'capital_loss', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'bodily_injuries', 'witnesses',
       'total_claim_amount', 'injury_claim', 'property_claim', 'vehicle_claim',
       'policy_csl_250_500', 'policy_csl_500_1000', 'insured_sex_MALE',
       'insured_education_level_College',
       'insured_education_level_High_School', 'insured_education_level_JD',
       'insured_education_level_MD', 'insured_education_level_Masters',
       'insured_education_level_PhD', 'insured_occupation_armed_forces',
       'insured_occupation_craft_repair', 'insured_occupation_exec_managerial',
       'insured_occupation_farming_fishing',
       'insured_occupation_handlers_cleaners',
       'insured_occupation_machine_op_inspct',
       'insured_occupation_other_service',
       'insured_occupation_priv_house_serv',
       'insured_occupati

In [46]:
y_pred_series = pd.Series(y_pred)

In [47]:
y_pred

array(['No Fraud Reported', 'No Fraud Reported', 'No Fraud Reported', ...,
       'No Fraud Reported', 'No Fraud Reported', 'No Fraud Reported'],
      dtype=object)

In [48]:
y_pred_series

0        No Fraud Reported
1        No Fraud Reported
2        No Fraud Reported
3        No Fraud Reported
4        No Fraud Reported
               ...        
58002       Fraud Reported
58003       Fraud Reported
58004    No Fraud Reported
58005    No Fraud Reported
58006    No Fraud Reported
Length: 58007, dtype: object

In [49]:
type(X_train), type(y_train), type(y_pred_series)

(pandas.core.frame.DataFrame,
 pandas.core.series.Series,
 pandas.core.series.Series)

In [50]:
y_pred_series.index

RangeIndex(start=0, stop=58007, step=1)

In [51]:
X_train_df = pd.DataFrame(X_train)
#y_train_df = pd.DataFrame(y_train)
y_train_df = pd.DataFrame({'target':y_train.index, 'value':y_train.values})

X_test_df = pd.DataFrame(X_test)
#y_test_df = pd.DataFrame(y_test)
y_test_df = pd.DataFrame({'target':y_test.index, 'value':y_test.values})

#y_pred_df = pd.DataFrame(y_pred)
y_pred_df = pd.DataFrame({'target':y_pred_series.index, 'value':y_pred_series.values})

In [52]:
type(X_train_df), type(y_train_df), type(X_test_df), type(y_test_df), type(y_pred_df)

(pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame)

In [53]:
X_train_df.name = 'X_train_df' 
y_train_df.name = 'y_train_df' 
X_test_df.name = 'X_test_df' 
y_test_df.name = 'y_test_df' 
y_pred_df.name = 'y_pred_df'

In [54]:
X_train_df.columns.name = 'X_train_df' 
y_train_df.columns.name = 'y_train_df' 
X_test_df.columns.name = 'X_test_df' 
y_test_df.columns.name = 'y_test_df' 
y_pred_df.columns.name = 'y_pred_df'

In [55]:
## registering the model in Fosfor Insight Designer.
register_model(
    model_obj=dtc, 
    session=my_session,
    x_train=X_train_df,
    y_train=y_train_df,
    x_test=X_test_df,
    y_test=y_test_df,
    y_pred=y_pred_df,
    source="Notebook",
    dataset_name="AUTO_INSURANCE_CLAIMS_DATA",
    dataset_source="Snowflake",
    #dataset_source="InMemory",
    name="Claims_Fraud_DTree_Classifier",
    description="Insurance claims model trained for Fraud Classification",
    flavour="sklearn",
    model_type="classification",
    conda_dependencies=["fosforml==1.1.6"]
)

"Failed to get dataset details. 'DataFrame' object has no attribute 'name'"

In [ ]:
designer

# Version 2 of Decision Tree Fraud Classifier Model

In [125]:
dtc = DecisionTreeClassifier(random_state=7)
dtc.fit(X_train, y_train)

DecisionTreeClassifier(random_state=7)

In [126]:
y_pred = dtc.predict(X_test)
y_prob = dtc.predict_proba(X_test)[:,1]

In [127]:
dtc_train_acc = accuracy_score(y_train, dtc.predict(X_train))
dtc_test_acc = accuracy_score(y_test, y_pred)

print(f"Training accuracy of Decision Tree is : {dtc_train_acc}")
print(f"Test accuracy of Decision Tree is : {dtc_test_acc}")

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Training accuracy of Decision Tree is : 1.0
Test accuracy of Decision Tree is : 0.9941263851657076
[[ 8305   156]
 [  188 49918]]
                   precision    recall  f1-score   support

   Fraud Reported       0.98      0.98      0.98      8461
No Fraud Reported       1.00      1.00      1.00     50106

         accuracy                           0.99     58567
        macro avg       0.99      0.99      0.99     58567
     weighted avg       0.99      0.99      0.99     58567



In [128]:
## registering the model in Fosfor Insight Designer using same name.
model_reg = register_model(dtc, 
               score, 
               name="Claims_Fraud_DTree_Classifier", 
               description="Insurance claim's model trained for Fraud Classification",
               flavour=MLModelFlavours.sklearn,
               model_type="classification",
               init_script="snowflake-connector-python[pandas]",
               y_true=y_test,
               y_pred=y_pred, 
               prob=y_prob,
               features=X_train.columns,
               input_type="json", 
               explain_ai=True,
               x_train=X_train, 
               x_test=X_test, 
               y_train=y_train.tolist(),
               y_test=y_test.tolist(),
               feature_names=X_train.columns.tolist(),
               original_features=X_train.columns.tolist(),
               feature_ids=X_train.columns,
               kyd=True, kyd_score = True)

Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%
